### Instructions on how to transform a single file
Install the required libraries in an isolated environment

In [1]:
import xarray
import re
from datetime import datetime

In [3]:
# Read a file of the new dataset
name = '/Users/vgaur/Downloads/CTCH4_methane_emis_1998.nc'
ds = xarray.open_dataset(name) # open the file
print('The netcdf file contains information as shown below')
ds

The netcdf file contains information as shown below


<xarray.Dataset>
Dimensions:               (latitude: 180, longitude: 360, months: 12)
Coordinates:
  * latitude              (latitude) float64 -89.5 -88.5 -87.5 ... 88.5 89.5
  * longitude             (longitude) float64 -179.5 -178.5 ... 178.5 179.5
  * months                (months) int32 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    fossil                (months, latitude, longitude) float64 ...
    fossil_global_sum     (months) float64 ...
    microbial             (months, latitude, longitude) float64 ...
    microbial_global_sum  (months) float64 ...
    pyrogenic             (months, latitude, longitude) float64 ...
    pyrogenic_global_sum  (months) float64 ...
    total                 (months, latitude, longitude) float64 ...
    total_global_sum      (months) float64 ...
    surface_area          (latitude, longitude) float64 ...
Attributes:
    email:        carbontracker.team@noaa.gov
    url:          http://gml.noaa.gov/ccgg/carbontracker-ch4/
    institution:  NOAA Global Monitoring Laboratory
    version:      CTCH4 v2023 1x1 monthly fluxes as of 08-Nov-2024 01:58:47 UTC

In [4]:
# Explore the netCDF file
print('Data valriables in the netCDF file are', ds.data_vars)
print('Dimensions mentioned in the netCDF file are', ds.dims)
#..... explore the netCDf file for more information

Data valriables in the netCDF file are Data variables:
    fossil                (months, latitude, longitude) float64 ...
    fossil_global_sum     (months) float64 ...
    microbial             (months, latitude, longitude) float64 ...
    microbial_global_sum  (months) float64 ...
    pyrogenic             (months, latitude, longitude) float64 ...
    pyrogenic_global_sum  (months) float64 ...
    total                 (months, latitude, longitude) float64 ...
    total_global_sum      (months) float64 ...
    surface_area          (latitude, longitude) float64 ...
Dimensions mentioned in the netCDF file are Frozen({'latitude': 180, 'longitude': 360, 'months': 12})


In [5]:
# once explored, look at the latitude, longitude and other values and comvert them if needed.
ds = ds.rename({"latitude": "lat", "longitude": "lon"})
ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby("lon")
variable = [var for var in ds.data_vars]

In [ ]:
# Parse through the time variable to extract single timestamp and create a dataarray for the given timestamp.
for time_increment in range(0, len(ds.months)):
    # split the original file name to create a unique COG name
    filename = name.split("/ ")[-1]
    filename_elements = re.split("[_ .]", filename)
    start_time = datetime(int(filename_elements[-2]), time_increment + 1, 1)
    for var in variable: # Parse through the data variables
        data = getattr(ds.isel(months=time_increment), var)
        data = data.isel(lat=slice(None, None, -1))
        data.rio.set_spatial_dims("lon", "lat", inplace=True)
        data.rio.write_crs("epsg:4326", inplace=True)

        # # insert date of generated COG into filename
        filename_elements.pop()
        filename_elements[-1] = start_time.strftime("%Y%m")
        filename_elements.insert(2, var)
        cog_filename = "_".join(filename_elements)
        # # add extension
        cog_filename = f"{cog_filename}.tif"

In [9]:
variable = [var for var in ds.data_vars if "global" not in var]

for time_increment in range(0, len(ds.months)):
    filename = name.split("/")[-1]
    filename_elements = re.split("[_ .]", filename)
    start_time = datetime(int(filename_elements[-2]), time_increment + 1, 1)
    for var in variable:
        filename = name.split("/")[-1]
        filename_elements = re.split("[_ .]", filename)
        data = getattr(ds.isel(months=time_increment), var)
        data = data.isel(lat=slice(None, None, -1))
        data = data.where(data != -9999, -9999)
        data.rio.set_spatial_dims("lon", "lat", inplace=True)
        data.rio.write_crs("epsg:4326", inplace=True)
        data.rio.write_nodata(-9999, inplace=True)

        # # insert date of generated COG into filename
        filename_elements.pop()
        filename_elements[-1] = start_time.strftime("%Y%m")
        filename_elements.insert(2, var)
        cog_filename = "_".join(filename_elements)
        # # add extension
        cog_filename = f"{cog_filename}.tif"

CTCH4_methane_fossil_emis_199801.tif
CTCH4_methane_microbial_emis_199801.tif
CTCH4_methane_pyrogenic_emis_199801.tif
CTCH4_methane_total_emis_199801.tif
CTCH4_methane_surface_area_emis_199801.tif
CTCH4_methane_fossil_emis_199802.tif
CTCH4_methane_microbial_emis_199802.tif
CTCH4_methane_pyrogenic_emis_199802.tif
CTCH4_methane_total_emis_199802.tif
CTCH4_methane_surface_area_emis_199802.tif
CTCH4_methane_fossil_emis_199803.tif
CTCH4_methane_microbial_emis_199803.tif
CTCH4_methane_pyrogenic_emis_199803.tif
CTCH4_methane_total_emis_199803.tif
CTCH4_methane_surface_area_emis_199803.tif
CTCH4_methane_fossil_emis_199804.tif
CTCH4_methane_microbial_emis_199804.tif
CTCH4_methane_pyrogenic_emis_199804.tif
CTCH4_methane_total_emis_199804.tif
CTCH4_methane_surface_area_emis_199804.tif
CTCH4_methane_fossil_emis_199805.tif
CTCH4_methane_microbial_emis_199805.tif
CTCH4_methane_pyrogenic_emis_199805.tif
CTCH4_methane_total_emis_199805.tif
CTCH4_methane_surface_area_emis_199805.tif
CTCH4_methane_fossil

## Validate the COG using QGIS or any other means.
Once validated, follow the steps mentioned in the README in the folder `data_transformation_plugins`